In [10]:
import importlib
from typing import Sequence
from datasets.dataset_dict import DatasetDict
import fire
import datasets
import sys

sys.path.append("/home/meissner/shortcut-pruning")
import numpy as np
from transformers import AutoTokenizer
from pytorch_lightning import LightningModule
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from tqdm import tqdm
import torch 
from src.models.hf_model import SequenceClassificationTransformer
from src.models.hf_model_pruned import PruningTransformer

In [22]:
from torch import softmax

In [11]:
baseline_checkpoint = "/remote/csifs1/disk0/meissner/shortcut-pruning/experiments/baselines/bert/mnli/multiruns/2022-04-18/13-07-56/0/checkpoints/last.ckpt"

In [19]:
model_names = [
    # "baseline", 
    # "undertrain", 
    # "tinybert", 
    # "earlystop", 
    # "ensemble_v3_max",
]
pruned_names = [
    "pruning_tiny",
    "pruning_utama_under",
    "pruning_stop160",
]
checkpoint_paths = [
    # "/remote/csifs1/disk0/meissner/shortcut-pruning/experiments/debiasing/undertrain/bert/mnli/runs/2022-04-18/21-33-11/checkpoints/last.ckpt",
    # "/remote/csifs1/disk0/meissner/shortcut-pruning/experiments/debias/with_tiny/bert/mnli/runs/2022-05-25/18-31-33/checkpoints/last.ckpt",
    # "/remote/csifs1/disk0/meissner/shortcut-pruning/experiments/debias/new_undertrain_with_limit/bert/mnli/runs/2022-04-27/15-35-33/checkpoints/last.ckpt",
    # "/remote/csifs1/disk0/meissner/shortcut-pruning/experiments/debias/ensemble_v3_max_scaled/bert/mnli/multiruns/2022-05-25/20-32-15/0/checkpoints/last.ckpt",
]
pruned_checkpoint_paths = [
    "/remote/csifs1/disk0/meissner/shortcut-pruning/experiments/pruning/with_tiny/vanilla_then_prune_v2/runs/2022-05-28/09-56-29/checkpoints/last.ckpt",
    "/remote/csifs1/disk0/meissner/shortcut-pruning/experiments/pruning/vanilla_then_prune/runs/2022-05-24/11-35-12/checkpoints/last.ckpt",
    "/remote/csifs1/disk0/meissner/shortcut-pruning/experiments/pruning/with_stop_160/vanilla_then_prune_v2/runs/2022-05-30/15-12-34/checkpoints/last.ckpt",
]

In [14]:
negation_words = ["no", "not", "don't", "none", "nothing", 
                  "never", "aren’t", "isn’t", "weren’t", 
                  "neither", "don’t", "didn’t", "doesn’t", 
                  "cannot", "hasn’t", "won’t"]

In [26]:
baseline_model = SequenceClassificationTransformer.load_from_checkpoint(baseline_checkpoint)
baseline_model.eval()
tokenizer = AutoTokenizer.from_pretrained(baseline_model.hparams.huggingface_model)
baseline_model.to("cuda")
print(model_name)
for word in negation_words:
    prem_logits = []
    hyp_logits = []
    input1 = tokenizer(word, "", return_tensors='pt').to("cuda")
    input2 = tokenizer("", word, return_tensors='pt').to("cuda")
    output1 = torch.softmax(model(input1)[0][0], -1).detach().cpu().tolist()[2]
    output2 = torch.softmax(model(input2)[0][0], -1).detach().cpu().tolist()[2]
    prem_logits.append(output1)
    hyp_logits.append(output2)
print(np.mean(prem_logits), np.mean(hyp_logits))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pruning_stop160
0.23411992192268372 0.6482416391372681


In [27]:
# Pruned models
for model_name, model_checkpoint in zip(pruned_names, pruned_checkpoint_paths):
    model = PruningTransformer.load_from_checkpoint(model_checkpoint)
    model.eval()
    model.to("cuda")
    print(model_name)
    for word in negation_words:
        prem_logits = []
        hyp_logits = []
        input1 = tokenizer(word, "", return_tensors='pt').to("cuda")
        input2 = tokenizer("", word, return_tensors='pt').to("cuda")
        output1 = torch.softmax(model(input1)[0][0], -1).detach().cpu().tolist()[2]
        output2 = torch.softmax(model(input2)[0][0], -1).detach().cpu().tolist()[2]
        prem_logits.append(output1)
        hyp_logits.append(output2)
    print(np.mean(prem_logits), np.mean(hyp_logits))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Could not compute total_training_steps, returninng -1 instead.
This should only happen if you are evaluating with your own prediction loop.
Make sure to se

pruning_tiny
0.20935477316379547 0.39354830980300903


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Could not compute total_training_steps, returninng -1 instead.
This should only happen if you are evaluating with your own prediction loop.
Make sure to se

pruning_utama_under
0.0076430244371294975 0.12544234097003937


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Could not compute total_training_steps, returninng -1 instead.
This should only happen if you are evaluating with your own prediction loop.
Make sure to se

pruning_stop160
0.23411992192268372 0.6482416391372681
